In [1]:
import pandas as pd
import numpy as np
import pickle
import random
import os

import keras
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D, Embedding
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
maxlen = 10
batch_size = 32
embedding_dims = 100
hidden_dims = 250
epochs = 500

In [0]:
def get_data(X_f, y_f):
    with open(X_f, 'rb') as f:
#         unpickler = pickle.Unpickler(f)
#         X = unpickler.load()
        X = pickle.load(f)
    with open(y_f, 'rb') as f:
        y = pickle.load(f)
    X = list(sum(X, []))
    y = list(sum(y, []))
    return X,y

def save_np(f, inp):
    np.save(f, inp)

def load_np(f):
    return np.load(f)

def gen_vocab(X):
    vocab = []
    for i in X:
        for j in i:
            vocab.append(j)
    return np.array(list(set(vocab)))

def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

def encode_text(tokenizer, lines, length):
    encoded = tokenizer.texts_to_sequences(lines)
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [5]:
X, y = get_data('./X_2.pkl', './y_2.pkl')
# X = pad_seq(X)
X = np.array(X)
X = encode_text(create_tokenizer(X), X, maxlen)
y = np.array(y).reshape((-1,1))
# save_np('X_2_emb.npy', X)
# save_np('y_2_emb.npy', y)

# X = load_np('X_2_emb.npy')
# y = load_np('y_2_emb.npy')

print(X.shape,y.shape)

vocab = gen_vocab(X)
print(vocab.shape[0])

(90796, 10) (90796, 1)
72115


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(63557, 10) (27239, 10) (63557, 1) (27239, 1)


In [7]:
model = Sequential()
model.add(Embedding(vocab.shape[0], embedding_dims,input_length=maxlen))
model.add(Conv1D(64, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()





Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 100)           7211500   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 6, 64)             32064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 3, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 192)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               24704     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 7,268,397
Trainable params: 7,268,397
Non-trainable params: 0
________________________________________

In [8]:
model.compile(loss='binary_crossentropy',
            optimizer=keras.optimizers.Adam(),
            metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
model.fit(X_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_data=(X_test, y_test)
    )




Train on 63557 samples, validate on 27239 samples
Epoch 1/500





63557/63557 [==============================] - 31s 484us/step - loss: 0.0371 - acc: 2.6748e-04 - val_loss: -0.0250 - val_acc: 0.0000e+00
Epoch 2/500
63557/63557 [==============================] - 28s 448us/step - loss: 0.0177 - acc: 0.0000e+00 - val_loss: -0.0150 - val_acc: 0.0000e+00
Epoch 3/500
63557/63557 [==============================] - 29s 450us/step - loss: 0.0115 - acc: 0.0000e+00 - val_loss: -0.0054 - val_acc: 0.0000e+00
Epoch 4/500
63557/63557 [==============================] - 29s 449us/step - loss: 0.0079 - acc: 4.7202e-05 - val_loss: 0.0162 - val_acc: 0.0000e+00
Epoch 5/500
63557/63557 [==============================] - 28s 447us/step - loss: 0.0027 - acc: 2.0454e-04 - val_loss: 0.0020 - val_acc: 0.0000e+00
Epoch 6/500
63557/63557 [==============================] - 29s 450us/step - loss: 0.0114 - acc: 0.0014 - val_loss: 0.0939 - val_acc: 0.0000e+00
Epoch 7/500
63557/63557 [==============================

In [0]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])